In [1]:
import math

import matplotlib.pyplot as plt
import itertools
import csv
import numpy as np
from pylab import *
import math
import pandas as pd
import glob
from sklearn.cluster import KMeans

from matplotlib import cm
import scipy.interpolate as interp
from mpl_toolkits.mplot3d import Axes3D

THIS IS THE ENSEMBLE PROCESSING ON THE DATA FROM THE SUBMISSION FROM SATURDAY 13.
POSTPROCESSING

exp0 and 1 and 10 are probably just one cluster



exp2 Ensemble replace trap cluster with 0,0
exp2 Traj replace trap cluster with the additional condition that alpha<0.25 with 0,0

exp3 probably confined Ensemble replace trap cluster with 0,0 not sure
exp3 Traj replace trap cluster with the additional condition that k<0.25 (or similar) with 0,0 not sure



exp 5 looks like 1 cluster but has a lot of CP

6 is just one?

exp4, 7 and  11 have likely three states

PARTS OF THIS IS BASED ON COUNTING THE CP, WHICH IS DONE IN THE CELL BELOW HERE

In [2]:
N_EXP = 12 # number of experiments
N_FOVS = 30 # number of fields of view per experiment

# Single-trajectory predictions dataset filepath, where results from ChallengePredictions.ipynb are stored.
path_track = "track_2/"

# note this notebook saves pdf's and npy's in the current directory.

In [3]:
# COUNTING CHANGEPOINTS (LONG WAY BUT IT WORKS)
import os
for exp in range(N_EXP):
# for exp in range(0,1):

    print("exp "+str(exp))
    # Define the directory where the files are stored
    directory = path_track+'exp_'+str(exp)

    # Initialize an empty list to store the arrays
    arrays = []

    aa = []

    for fovIdx in range(N_FOVS):
#         print("fov "+str(fovIdx))

    #     print(fovIdx)
        filename = f'fov_{fovIdx}.txt'
    #     print(filename)
        filepath = os.path.join(directory, filename)
    #     print(filepath)
        with open(filepath) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            for row in csv_reader:
    #           
                aa.append([float(row[i]) for i in range(len(row)) if ((i)%4==2 and i!=0)]) 
#     print(len(aa))

    af = np.array(list(itertools.chain.from_iterable(aa)))

    print("n traj",len(aa),"n segments",len(af))
 

exp 0
n traj 999 n segments 1006
exp 1
n traj 1007 n segments 1014
exp 2
n traj 1394 n segments 2312
exp 3
n traj 1170 n segments 2100
exp 4
n traj 6437 n segments 14905
exp 5
n traj 1333 n segments 3157
exp 6
n traj 1299 n segments 2294
exp 7
n traj 8767 n segments 27397
exp 8
n traj 1232 n segments 1413
exp 9
n traj 1263 n segments 2417
exp 10
n traj 1556 n segments 1630
exp 11
n traj 1812 n segments 9543


In [ ]:
# for experiment 0,1, and 10 we believe there is a single state so we don't cluster

import os
# import numpy as np
for exp in [0,1,10]:
# for exp in range(0,12):
# for exp in range(0,1):

    # Specify the number of clusters
    k_set = 1


    print("exp "+str(exp))
    # Define the directory where the files are stored
    directory = path_track+'exp_'+str(exp)

    # Initialize an empty list to store the arrays
    arrays = []
    # Ys = []
    kk = []
    aa = []
    tt = []
    for fovIdx in range(N_FOVS):
        print("fov "+str(fovIdx))

    #     print(fovIdx)
        filename = f'fov_{fovIdx}.txt'
    #     print(filename)
        filepath = os.path.join(directory, filename)
    #     print(filepath)
        with open(filepath) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            for row in csv_reader:
    #             Ys.append(np.array([float(i) for i in row[1:]])) # K, a, 2.0, length (K and a in reverse order as ensemble label..)
                kk.append([float(row[i]) for i in range(len(row)) if ((i)%4==1 and i!=0)]) 
        
                aa.append([float(row[i]) for i in range(len(row)) if ((i)%4==2 and i!=0)]) 
                tt.append([float(row[i]) for i in range(len(row)) if ((i)%4==0 and i!=0)]) 

    af = np.array(list(itertools.chain.from_iterable(aa)))
    kf = np.array(list(itertools.chain.from_iterable(kk)))
#     tf = np.array(list(itertools.chain.from_iterable(tt)))

    if  np.isnan(af).any()== True :

        print("nan alpha at exp ",exp, "in pos",np.where(np.isnan(af)))
#     if  np.isnan(kf).any()== True :

#         print("nan k at exp ",exp, "at fov",fovIdx)
    # Extracting the length of each segment
    af = np.nan_to_num(af, copy=True, nan=0.001, posinf=None, neginf=None)
    kf = np.nan_to_num(kf, copy=True, nan=0.001, posinf=None, neginf=None)

    
    tdif = []
    for i in range(len(tt)):
        td = np.diff(tt[i]) # difference between successive change points
        tf = tt[i][0]       #The first entry in tt[i] is the length of the first segment
        ts = np.array(np.append(tf,td)) # creating the vector of segment lenght for trajectory i
        tdif.append(ts)
    tdf = np.array(list(itertools.chain.from_iterable(tdif)))    #flattening the list into a 1d array
    if  np.isnan(tdf).any()== True :

        print("nan t at exp ",exp, "at fov",fovIdx)


    tdf = np.nan_to_num(tdf, copy=True, nan=0.001, posinf=None, neginf=None)

    # Example arrays of coordinates
    x = af
    y = kf
    
#     If you want to use time uncomment z
#     z = tdf
#     coordinates = np.column_stack((x, y, z))

    # Combine the x and y coordinates into a single array
    coordinates = np.column_stack((x, y))
#     coordinates = np.nan_to_num(coordinates, copy=True, nan=0.001, posinf=None, neginf=None)


    # Create the KMeans model
    kmeans = KMeans(n_clusters=k_set, random_state=0, n_init =30)

    # Fit the model to the data
    kmeans.fit(coordinates)

    # Predict the cluster for each data point
    # labels = kmeans.predict(coordinates)
    labels = kmeans.labels_

    # Get the cluster centers
    centers = kmeans.cluster_centers_

    # Print the cluster labels and centers
    print("Cluster Labels:", labels)
    print("Cluster Centers:\n", centers)
    fig = plt.figure()# Plot the results
    plt.scatter(coordinates[:, 0], coordinates[:, 1], c=labels, cmap='viridis', marker='o', s =0.2)
    plt.scatter(centers[:, 0], centers[:, 1], c='red', marker='x')
    plt.xlabel('$\\alpha$')
    plt.ylabel('K')
    plt.xlim(0,2)

    plt.title('K-Means Clustering for exp'+str(exp))
    plt.show()
    fig.savefig('exp_'+str(exp)+'.pdf')
    
    
    fig = plt.figure()# Plot the results
    plt.scatter(coordinates[:, 0], coordinates[:, 1], c=labels, cmap='viridis', marker='o', s =0.2)
    plt.scatter(centers[:, 0], centers[:, 1], c='red', marker='x')
    plt.xlabel('$\\alpha$')
    plt.ylabel('K')
    plt.xlim(0,2)
    plt.ylim(0.001,16)
    plt.title('K-Means Clustering for exp'+str(exp))
    plt.yscale('log')
    plt.show()
    fig.savefig('exp_'+str(exp)+'_log.pdf')
    
#     mma = [np.mean(af[labels==0]),np.mean(af[labels==1])]
#     stda = [np.std(af[labels==0]),np.std(af[labels==1])]

#     mmk = [np.mean(kf[labels==0]),np.mean(kf[labels==1])]
#     stdk = [np.std(kf[labels==0]),np.std(kf[labels==1])]

#     np.save("mean_a"+str(exp),mma)
#     np.save("std_a"+str(exp),stda)

#     np.save("mean_k"+str(exp),mmk)
#     np.save("std_k"+str(exp),stdk)

    
    # The time spent in a cluster is the sum of the length of the duration of all the segments belonging to a cluster
#     tdf_lab = []
    tdf_tot = []
#     tdf_mean = []
    mma = []
    stda = []
    mmk = []
    stdk = []
    for i in range(k_set):
    #     print(i)
        tdf_l = tdf[labels == i] #duration for segments in class i
        a_l = af[labels == i] #alpha for segments in class i
        k_l = kf[labels == i] #k for segments in class i

        mma.append(np.mean(a_l)) #mean alpha for segments in class i
        stda.append(np.std(a_l)) #std of alpha for segments in class i
        
        mmk.append(np.mean(k_l)) #mean k for segments in class i
        stdk.append(np.std(k_l)) #std of k for segments in class i
        
#         tdf_lab.append(tdf_l)
        tdf_tot.append(np.sum(tdf_l)) # total time spent in class i
#         tdf_mean.append(np.mean(tdf_l)) 

        
    np.save("mean_a"+str(exp),mma)
    np.save("std_a"+str(exp),stda)

    np.save("mean_k"+str(exp),mmk)
    np.save("std_k"+str(exp),stdk)    
    np.save("std_k"+str(exp),stdk)    

#     Normalising the amount of time spent in each state
    tdf_norm = np.array(tdf_tot)
#     print(tdf_norm)
    tot_t = np.sum(tdf_norm)
#     print(tot_t)
    tdf_norm = tdf_norm/tot_t
#     print(tdf_norm)
    np.save("time"+str(exp),tdf_norm)    
    
        
    
    print("mean $\\alpha$",mma,"\n","mean K",mmk,"\n","std $\\alpha$",stda,"\n","std k",stdk,"\n","time",tdf_norm)
    
    inertia = []
    k_range = range(1, 6)
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=0, n_init =30)
        kmeans.fit(coordinates)
        inertia.append(kmeans.inertia_)

    # Plot the Elbow Method
    fig2 = plt.figure(figsize=(8, 5))
    plt.plot(k_range, inertia, 'bo-')
    plt.xlabel('Number of clusters (k)')
    plt.ylabel('Inertia (Sum of squared distances)')
    plt.title('Elbow Method For Optimal k for exp'+str(exp))
    plt.show()
    fig2.savefig('elbow_exp_'+str(exp)+'.pdf')

   


In [ ]:
# for experiment 4,7, and 11 we believe there are three states so we do look for 3 clusters

import os
# import numpy as np
for exp in [4,7,11]:
# for exp in range(0,12):
# for exp in range(0,1):

    # Specify the number of clusters
    k_set = 3


    print("exp "+str(exp))
    # Define the directory where the files are stored
    directory = path_track+'exp_'+str(exp)

    # Initialize an empty list to store the arrays
    arrays = []
    # Ys = []
    kk = []
    aa = []
    tt = []
    for fovIdx in range(N_FOVS):
        print("fov "+str(fovIdx))

    #     print(fovIdx)
        filename = f'fov_{fovIdx}.txt'
    #     print(filename)
        filepath = os.path.join(directory, filename)
    #     print(filepath)
        with open(filepath) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            for row in csv_reader:
    #             Ys.append(np.array([float(i) for i in row[1:]])) # K, a, 2.0, length (K and a in reverse order as ensemble label..)
                kk.append([float(row[i]) for i in range(len(row)) if ((i)%4==1 and i!=0)]) 
        
                aa.append([float(row[i]) for i in range(len(row)) if ((i)%4==2 and i!=0)]) 
                tt.append([float(row[i]) for i in range(len(row)) if ((i)%4==0 and i!=0)]) 

    af = np.array(list(itertools.chain.from_iterable(aa)))
    kf = np.array(list(itertools.chain.from_iterable(kk)))
#     tf = np.array(list(itertools.chain.from_iterable(tt)))

    if  np.isnan(af).any()== True :

        print("nan alpha at exp ",exp, "in pos",np.where(np.isnan(af)))
#     if  np.isnan(kf).any()== True :

#         print("nan k at exp ",exp, "at fov",fovIdx)
    # Extracting the length of each segment
    af = np.nan_to_num(af, copy=True, nan=0.001, posinf=None, neginf=None)
    kf = np.nan_to_num(kf, copy=True, nan=0.001, posinf=None, neginf=None)

    
    tdif = []
    for i in range(len(tt)):
        td = np.diff(tt[i]) # difference between successive change points
        tf = tt[i][0]       #The first entry in tt[i] is the length of the first segment
        ts = np.array(np.append(tf,td)) # creating the vector of segment lenght for trajectory i
        tdif.append(ts)
    tdf = np.array(list(itertools.chain.from_iterable(tdif)))    #flattening the list into a 1d array
    if  np.isnan(tdf).any()== True :

        print("nan t at exp ",exp, "at fov",fovIdx)


    tdf = np.nan_to_num(tdf, copy=True, nan=0.001, posinf=None, neginf=None)

    # Example arrays of coordinates
    x = af
    y = kf
    
#     If you want to use time uncomment z
#     z = tdf
#     coordinates = np.column_stack((x, y, z))

    # Combine the x and y coordinates into a single array
    coordinates = np.column_stack((x, y))
#     coordinates = np.nan_to_num(coordinates, copy=True, nan=0.001, posinf=None, neginf=None)


    # Create the KMeans model
    kmeans = KMeans(n_clusters=k_set, random_state=0, n_init =30)

    # Fit the model to the data
    kmeans.fit(coordinates)

    # Predict the cluster for each data point
    # labels = kmeans.predict(coordinates)
    labels = kmeans.labels_

    # Get the cluster centers
    centers = kmeans.cluster_centers_

    # Print the cluster labels and centers
    print("Cluster Labels:", labels)
    print("Cluster Centers:\n", centers)
    fig = plt.figure()# Plot the results
    plt.scatter(coordinates[:, 0], coordinates[:, 1], c=labels, cmap='viridis', marker='o', s =0.2)
    plt.scatter(centers[:, 0], centers[:, 1], c='red', marker='x')
    plt.xlabel('$\\alpha$')
    plt.ylabel('K')
    plt.xlim(0,2)

    plt.title('K-Means Clustering for exp'+str(exp))
    plt.show()
    fig.savefig('exp_'+str(exp)+'.pdf')
    
    
    fig = plt.figure()# Plot the results
    plt.scatter(coordinates[:, 0], coordinates[:, 1], c=labels, cmap='viridis', marker='o', s =0.2)
    plt.scatter(centers[:, 0], centers[:, 1], c='red', marker='x')
    plt.xlabel('$\\alpha$')
    plt.ylabel('K')
    plt.xlim(0,2)
    plt.ylim(0.001,16)
    plt.title('K-Means Clustering for exp'+str(exp))
    plt.yscale('log')
    plt.show()
    fig.savefig('exp_'+str(exp)+'_log.pdf')
    
 
    # The time spent in a cluster is the sum of the length of the duration of all the segments belonging to a cluster
#     tdf_lab = []
    tdf_tot = []
#     tdf_mean = []
    mma = []
    stda = []
    mmk = []
    stdk = []
    for i in range(k_set):
    #     print(i)
        tdf_l = tdf[labels == i] #duration for segments in class i
        a_l = af[labels == i] #alpha for segments in class i
        k_l = kf[labels == i] #k for segments in class i

        mma.append(np.mean(a_l)) #mean alpha for segments in class i
        stda.append(np.std(a_l)) #std of alpha for segments in class i
        
        mmk.append(np.mean(k_l)) #mean k for segments in class i
        stdk.append(np.std(k_l)) #std of k for segments in class i
        
#         tdf_lab.append(tdf_l)
        tdf_tot.append(np.sum(tdf_l)) # total time spent in class i
#         tdf_mean.append(np.mean(tdf_l)) 

        
    np.save("mean_a"+str(exp),mma)
    np.save("std_a"+str(exp),stda)

    np.save("mean_k"+str(exp),mmk)
    np.save("std_k"+str(exp),stdk)    
    np.save("std_k"+str(exp),stdk)    

#     Normalising the amount of time spent in each state
    tdf_norm = np.array(tdf_tot)
#     print(tdf_norm)
    tot_t = np.sum(tdf_norm)
#     print(tot_t)
    tdf_norm = tdf_norm/tot_t
#     print(tdf_norm)
    np.save("time"+str(exp),tdf_norm)    
    
        
    
    print("mean $\\alpha$",mma,"\n","mean K",mmk,"\n","std $\\alpha$",stda,"\n","std k",stdk,"\n","time",tdf_norm)
    
    inertia = []
    k_range = range(1, 6)
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=0, n_init =30)
        kmeans.fit(coordinates)
        inertia.append(kmeans.inertia_)

    # Plot the Elbow Method
    fig2 = plt.figure(figsize=(8, 5))
    plt.plot(k_range, inertia, 'bo-')
    plt.xlabel('Number of clusters (k)')
    plt.ylabel('Inertia (Sum of squared distances)')
    plt.title('Elbow Method For Optimal k for exp'+str(exp))
    plt.show()
    fig2.savefig('elbow_exp_'+str(exp)+'.pdf')

   


In [ ]:
# for experiment the other ones 2,3,5,6,8,9 we believe there are two states so we do look for 2 clusters

import os
# import numpy as np
for exp in [2,3,5,6,8,9]:
# for exp in range(0,12):
# for exp in range(0,1):

    # Specify the number of clusters
    k_set = 2


    print("exp "+str(exp))
    # Define the directory where the files are stored
    directory = path_track+'exp_'+str(exp)

    # Initialize an empty list to store the arrays
    arrays = []
    # Ys = []
    kk = []
    aa = []
    tt = []
    for fovIdx in range(N_FOVS):
        print("fov "+str(fovIdx))

    #     print(fovIdx)
        filename = f'fov_{fovIdx}.txt'
    #     print(filename)
        filepath = os.path.join(directory, filename)
    #     print(filepath)
        with open(filepath) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            for row in csv_reader:
    #             Ys.append(np.array([float(i) for i in row[1:]])) # K, a, 2.0, length (K and a in reverse order as ensemble label..)
                kk.append([float(row[i]) for i in range(len(row)) if ((i)%4==1 and i!=0)]) 
        
                aa.append([float(row[i]) for i in range(len(row)) if ((i)%4==2 and i!=0)]) 
                tt.append([float(row[i]) for i in range(len(row)) if ((i)%4==0 and i!=0)]) 

    af = np.array(list(itertools.chain.from_iterable(aa)))
    kf = np.array(list(itertools.chain.from_iterable(kk)))
#     tf = np.array(list(itertools.chain.from_iterable(tt)))

    if  np.isnan(af).any()== True :

        print("nan alpha at exp ",exp, "in pos",np.where(np.isnan(af)))
#     if  np.isnan(kf).any()== True :

#         print("nan k at exp ",exp, "at fov",fovIdx)
    # Extracting the length of each segment
    af = np.nan_to_num(af, copy=True, nan=0.001, posinf=None, neginf=None)
    kf = np.nan_to_num(kf, copy=True, nan=0.001, posinf=None, neginf=None)

    
    tdif = []
    for i in range(len(tt)):
        td = np.diff(tt[i]) # difference between successive change points
        tf = tt[i][0]       #The first entry in tt[i] is the length of the first segment
        ts = np.array(np.append(tf,td)) # creating the vector of segment lenght for trajectory i
        tdif.append(ts)
    tdf = np.array(list(itertools.chain.from_iterable(tdif)))    #flattening the list into a 1d array
    if  np.isnan(tdf).any()== True :

        print("nan t at exp ",exp, "at fov",fovIdx)


    tdf = np.nan_to_num(tdf, copy=True, nan=0.001, posinf=None, neginf=None)

    # Example arrays of coordinates
    x = af
    y = kf
    
#     If you want to use time uncomment z
#     z = tdf
#     coordinates = np.column_stack((x, y, z))

    # Combine the x and y coordinates into a single array
    coordinates = np.column_stack((x, y))
#     coordinates = np.nan_to_num(coordinates, copy=True, nan=0.001, posinf=None, neginf=None)


    # Create the KMeans model
    kmeans = KMeans(n_clusters=k_set, random_state=0, n_init =30)

    # Fit the model to the data
    kmeans.fit(coordinates)

    # Predict the cluster for each data point
    # labels = kmeans.predict(coordinates)
    labels = kmeans.labels_

    # Get the cluster centers
    centers = kmeans.cluster_centers_

    # Print the cluster labels and centers
    print("Cluster Labels:", labels)
    print("Cluster Centers:\n", centers)
    fig = plt.figure()# Plot the results
    plt.scatter(coordinates[:, 0], coordinates[:, 1], c=labels, cmap='viridis', marker='o', s =0.2)
    plt.scatter(centers[:, 0], centers[:, 1], c='red', marker='x')
    plt.xlabel('$\\alpha$')
    plt.ylabel('K')
    plt.xlim(0,2)

    plt.title('K-Means Clustering for exp'+str(exp))
    plt.show()
    fig.savefig('exp_'+str(exp)+'.pdf')
    
    
    fig = plt.figure()# Plot the results
    plt.scatter(coordinates[:, 0], coordinates[:, 1], c=labels, cmap='viridis', marker='o', s =0.2)
    plt.scatter(centers[:, 0], centers[:, 1], c='red', marker='x')
    plt.xlabel('$\\alpha$')
    plt.ylabel('K')
    plt.xlim(0,2)
    plt.ylim(0.001,16)
    plt.title('K-Means Clustering for exp'+str(exp))
    plt.yscale('log')
    plt.show()
    fig.savefig('exp_'+str(exp)+'_log.pdf')
    
#     mma = [np.mean(af[labels==0]),np.mean(af[labels==1])]
#     stda = [np.std(af[labels==0]),np.std(af[labels==1])]

#     mmk = [np.mean(kf[labels==0]),np.mean(kf[labels==1])]
#     stdk = [np.std(kf[labels==0]),np.std(kf[labels==1])]

#     np.save("mean_a"+str(exp),mma)
#     np.save("std_a"+str(exp),stda)

#     np.save("mean_k"+str(exp),mmk)
#     np.save("std_k"+str(exp),stdk)

    
    # The time spent in a cluster is the sum of the length of the duration of all the segments belonging to a cluster
#     tdf_lab = []
    tdf_tot = []
#     tdf_mean = []
    mma = []
    stda = []
    mmk = []
    stdk = []
    for i in range(k_set):
    #     print(i)
        tdf_l = tdf[labels == i] #duration for segments in class i
        a_l = af[labels == i] #alpha for segments in class i
        k_l = kf[labels == i] #k for segments in class i

        mma.append(np.mean(a_l)) #mean alpha for segments in class i
        stda.append(np.std(a_l)) #std of alpha for segments in class i
        
        mmk.append(np.mean(k_l)) #mean k for segments in class i
        stdk.append(np.std(k_l)) #std of k for segments in class i
        
#         tdf_lab.append(tdf_l)
        tdf_tot.append(np.sum(tdf_l)) # total time spent in class i
#         tdf_mean.append(np.mean(tdf_l)) 

        
    np.save("mean_a"+str(exp),mma)
    np.save("std_a"+str(exp),stda)

    np.save("mean_k"+str(exp),mmk)
    np.save("std_k"+str(exp),stdk)    
    np.save("std_k"+str(exp),stdk)    

#     Normalising the amount of time spent in each state
    tdf_norm = np.array(tdf_tot)
#     print(tdf_norm)
    tot_t = np.sum(tdf_norm)
#     print(tot_t)
    tdf_norm = tdf_norm/tot_t
#     print(tdf_norm)
    np.save("time"+str(exp),tdf_norm)    
    
        
    
    print("mean $\\alpha$",mma,"\n","mean K",mmk,"\n","std $\\alpha$",stda,"\n","std k",stdk,"\n","time",tdf_norm)
    
    inertia = []
    k_range = range(1, 6)
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=0, n_init =30)
        kmeans.fit(coordinates)
        inertia.append(kmeans.inertia_)

    # Plot the Elbow Method
    fig2 = plt.figure(figsize=(8, 5))
    plt.plot(k_range, inertia, 'bo-')
    plt.xlabel('Number of clusters (k)')
    plt.ylabel('Inertia (Sum of squared distances)')
    plt.title('Elbow Method For Optimal k for exp'+str(exp))
    plt.show()
    fig2.savefig('elbow_exp_'+str(exp)+'.pdf')

   


SINCE EXP 2 IS CLEARLY TRAPPED WE ASSIGN A VERY SMALL MEAN AND AVERAGE TO BOTH K AND ALPHA, OVERWRITING THE WHAT THE CLUSTER SAID. THIS IS POTENTIALLY DANGEROUS

In [7]:
mean_a2 = np.load("mean_a2.npy")
mean_k2 = np.load("mean_k2.npy")
std_a2 = np.load("std_a2.npy")
std_k2 = np.load("std_k2.npy")
print("mean alpha",mean_a2,"mean K",mean_k2)
print("std alpha",std_a2,"std K",std_k2)

mean alpha [1.0067722  0.04544294] mean K [1.07781018 0.02494362]
std alpha [0.15590775 0.15399206] std K [0.36769337 0.09130023]


In [8]:
mean_a2 = np.array([mean_a2[0],0.01])
mean_k2 = np.array([mean_k2[0],0.0001])
std_a2 = np.array([std_a2[0],0.001])
std_k2 = np.array([std_k2[0],0.00001])
print("mean alpha",mean_a2,"mean K",mean_k2)
print("std alpha",std_a2,"std K",std_k2)


mean alpha [1.0067722 0.01     ] mean K [1.07781018e+00 1.00000000e-04]
std alpha [0.15590775 0.001     ] std K [3.67693371e-01 1.00000000e-05]


In [ ]:
np.save("mean_a2.npy",mean_a2)
np.save("mean_k2.npy",mean_k2)
np.save("std_a2.npy",std_a2)
np.save("std_k2.npy",std_k2)

In [ ]:
#Putting the oututp in the correct form from saved files

In [ ]:
Kvar, Kmean, avar, amean, times = [], [], [], [], []
for e in range(N_EXP):
    Kvar.append(np.load(f'std_k{e}.npy')**2)
    avar.append(np.load(f'std_a{e}.npy')**2)
    Kmean.append(np.load(f'mean_k{e}.npy'))
    amean.append(np.load(f'mean_a{e}.npy'))
    times.append(np.load(f'time{e}.npy'))

EnsembleDat = []
for e in range(N_EXP):
    expDat = np.ones((5, 2))
    expDat = [[],[],[],[],[]]
    expDat[0] = Kmean[e]
    expDat[1] = Kvar[e]
    expDat[2] = amean[e]
    expDat[3] = avar[e]
    expDat[4] = times[e]
    EnsembleDat.append(expDat)

In [ ]:
# make the ensemble labels files in the same directories as the single-traj labels

for e in range(N_EXP):
    path_exp = path_track+f"exp_{e}/"
    file_name = path_exp + 'ensemble_labels.txt'
    with open(file_name, 'a') as f:
        f.write(f'model: multi_state; num_state: {2} \n')
        np.savetxt(f, EnsembleDat[e], delimiter=';')

In [10]:
# Example of how to draw boundaries by hand, not used here
h0 = np.zeros(len(af))
h1 = np.ones(len(af))
hand_labels = np.where(((af <0.5) & (kf<0.3)), h0,h1)

In [ ]:
fig = plt.figure()# Plot the results
plt.scatter(coordinates[:, 0], coordinates[:, 1], c=hand_labels, cmap='viridis', marker='o', s =0.2)
plt.scatter(centers[:, 0], centers[:, 1], c='red', marker='x')
plt.xlabel('$\\alpha$')
plt.ylabel('K')
plt.xlim(0,2)
#     plt.ylim(0,4)
plt.title('K-Means Clustering for sub 2  exp'+str(exp))
plt.show()
#     fig.savefig('exp_'+str(exp)+'_T_T.pdf')
